In [1]:
%%configure -f
{"name": "sue-manga", "executorMemory": "12G", "numExecutors": 8, "executorCores": 2,
 "conf": {"spark.yarn.appMasterEnv.PYSPARK_PYTHON":"python3", "spark.sql.shuffle.partitions": 300}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
11,application_1602298274728_0012,pyspark,dead,Link,,


In [2]:
import sys
import os
import subprocess
from io import BytesIO
from gzip import GzipFile
from pyspark.sql import Row
import glob
import time
import numpy as np
from itertools import repeat
from collections import OrderedDict

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
12,application_1602298274728_0013,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
tmp = spark.read.parquet('hdfs:///manga/brian-test/dr15/v2_4_3/modelcubes')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
tmp.first()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(plateifu='8145-6104', mangaid='1-152567', ra=116.511679886, dec=28.878071563, daptype='HYB10-GAU-MILESHC', spaxel_idx=625, x=31, y=11, wave=3621.595947265625, flux=-0.012602979317307472, ivar=379.7341003417969, mask=32, model=0.0, emline=0.0, emline_base=0.0, emline_mask=128, binid_binned_spectra=60, binid_stellar_cont=60, binid_emline_moments=0, binid_emline_models=832, binid_spectral_indices=0)

In [23]:
maps = spark.read.parquet('hdfs:///manga/brian-test/dr15/v2_4_3/maps')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
tmp.printSchema()

In [ ]:
tmp.show(10)

In [ ]:
maps.printSchema()

In [26]:
maps.createOrReplaceTempView("maps")
maps_test = spark.sql("select * from maps limit 10")
maps_test.createOrReplaceTempView("maps_test")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
modelcubes = spark.read.parquet('hdfs:///manga/arik-test/dr15/v2_4_3/modelcubes_voxel')
modelcubes.createOrReplaceTempView('modelcubes')
modelcubes_test = spark.sql("select * from modelcubes limit 10")
modelcubes_test.createOrReplaceTempView("modelcubes_test")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
modelcubes = spark.read.parquet('hdfs:///manga/brian-test/dr15/v2_4_3/modelcubes')
modelcubes.createOrReplaceTempView('modelcubes')
modelcubes_test = spark.sql("select * from modelcubes limit 10")
modelcubes_test.createOrReplaceTempView("modelcubes_test")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
drpall = spark.read.parquet('hdfs:///manga/brian-test/dr15/v2_4_3/drpall')
drpall.createOrReplaceTempView('drpall')
drpall_test = spark.sql("select * from drpall limit 10")
drpall_test.createOrReplaceTempView("drpall_test")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
dapall = spark.read.parquet('hdfs:///manga/brian-test/dr15/v2_4_3/dapall')
dapall.createOrReplaceTempView('dapall')
dapall_test = spark.sql("select * from dapall limit 10")
dapall_test.createOrReplaceTempView("dapall_test")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
authfileDF =spark.read.json("hdfs:///.config/creds.json")

kvs = authfileDF.collect()[0]
jdbc_username = kvs.jdbc_username
jdbc_password = kvs.jdbc_password

#print(jdbc_username)
#print(jdbc_password)


In [4]:
#Write from Spark to SQL table using MSSQL Spark Connector
print("Use MSSQL connector to write to master SQL instance ")

servername = "jdbc:sqlserver://master-0.master-svc"
dbname = "manga_bdc"
url = servername + ";" + "databaseName=" + dbname + ";"

#dbtable = "Object" this is not needed
user = jdbc_username
password = jdbc_password # Please specify password here

datapool_table = "modelcubes_test"
#datapool_table = "Object"
datasource_name = "SqlDataPool"

batchsize = 1000000


try:
  modelcubes_test.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", datapool_table) \
    .option("user", user) \
    .option("password", password) \
    .option("dataPoolDataSource",datasource_name)\
    .option("batchsize",batchsize)\
    .save()
except ValueError as error :
    print("MSSQL Connector write failed", error)

print("MSSQL Connector write(overwrite) to data pool external table succeeded")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use MSSQL connector to write to master SQL instance 
MSSQL Connector write(overwrite) to data pool external table succeeded